In [1]:
############ library  as tf
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import re
from PIL import Image
import shutil
# import xml.etree.ElementTree as et
import random
import matplotlib.pyplot as plt


print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.0


In [4]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [5]:
class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes, finalAct="softmax"):
		# 인풋 이미지의 차원과, 채널에 해당하는 축을 설정하여 모델을 초기화합니다
		# "channels_last"는 채널의 축이 마지막에 오는 것을 의미합니다
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
 
 		# 만약 "channels_first"를 사용한다면, 인풋 이미지의 차원을
		# 그에 맞게 바꿔줍니다
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

    # CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))
  

    # (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
 
		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
  

    # FC => RELU
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
 
 		# 단일 라벨 분류는 *softmax* 활성화 함수를 사용합니다
		# 다중 라벨 분류는 *sigmoid* 활성화 함수를 사용합니다
		model.add(Dense(classes))
		model.add(Activation(finalAct))
 
 		# 네트워크 아키텍처를 반환합니다
		return model

In [ ]:
!pip install opencv

ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


In [31]:
import matplotlib
matplotlib.use("Agg")
 
# 필요한 패키지들을 가져옵니다
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
# from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unzip -qq "/content/drive/MyDrive/이게뭐약/AI/색깔모양분류/colorimg.zip"

In [ ]:
# # construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True, help="path to input dataset (i.e., directory of images)")
# ap.add_argument("-m", "--model", required=True, help="path to output model")
# ap.add_argument("-l", "--labelbin", required=True, help="path to output label binarizer")
# ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output accuracy/loss plot")
# args = vars(ap.parse_args())

In [10]:
import easydict
 
args = easydict.EasyDict({
        "dataset": True,
        "model": True,
        "labelbin": True,
        "plot": "plot.jpg"

})

In [29]:
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [12]:
# 이미지 경로를 섞어줍니다
print("[INFO] loading images...")
imagePaths = "/content/colorimg"
# random.seed(42)
# random.shuffle(imagePaths)
 
# 데이터와 라벨을 초기화합니다
data = []
labels = []

[INFO] loading images...


In [13]:
file_list = os.listdir(imagePaths)

In [14]:
file_list

['원형_하양-200612728.jpg',
 '장방형_빨강-198500049.jpg',
 '원형_노랑-200606617.jpg',
 '원형_하양-198700309.jpg',
 '장방형_하양-198300344.jpg',
 '장방형_하양-200702513.jpg',
 '원형_초록-200603872.jpg',
 '원형_하양-200000613.jpg',
 '원형_하양-200802220.jpg',
 '장방형_하양-199601061.jpg',
 '원형_하양-200410511.jpg',
 '장방형_연두-199600199.jpg',
 '사각형_노랑-200009063.jpg',
 '타원형_노랑-200400219.jpg',
 '장방형_하양-200400322.jpg',
 '원형_하양-200511125.jpg',
 '원형_하양-200410070.jpg',
 '장방형_갈색-198900528.jpg',
 '원형_하양-199701269.jpg',
 '타원형_노랑-200300286.jpg',
 '원형_노랑-199300959.jpg',
 '타원형_파랑-200300898.jpg',
 '장방형_하양-200600055.jpg',
 '장방형_파랑-199700942.jpg',
 '원형_하양-200009077.jpg',
 '장방형_연두-200410502.jpg',
 '원형_하양-199900349.jpg',
 '장방형_파랑-200000803.jpg',
 '원형_주황-200500476.jpg',
 '장방형_초록-200400143.jpg',
 '원형_주황-200702734.jpg',
 '타원형_노랑-200511380.jpg',
 '오각형_분홍-199700553.jpg',
 '타원형_하양-200300277.jpg',
 '장방형_갈색, 투명-200500431.jpg',
 '타원형_하양-200502773.jpg',
 '원형_하양-199800514.jpg',
 '장방형_빨강-200400372.jpg',
 '타원형_노랑-200600080.jpg',
 '장방형_하양-199602451.jpg',
 '장방형_분홍-199

In [27]:
file_list[9].split("-")[0].split("_")

['장방형', '하양']

In [42]:
# 인풋 이미지들에 대해 아래의 반복문을 수행합니다
for imagePath in file_list:
  image = cv2.imread('./colorimg/'+imagePath)
  # print(image)
  image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
  image = img_to_array(image)
  data.append(image)
 
 	# # # 이미지 경로에서 라벨을 추출한 후, 라벨 리스트를 업데이트합니다
  l = label = imagePath.split("-")[0].split("_")
  labels.append(l)

In [43]:
labels

[['원형', '하양'],
 ['장방형', '빨강'],
 ['원형', '노랑'],
 ['원형', '하양'],
 ['장방형', '하양'],
 ['장방형', '하양'],
 ['원형', '초록'],
 ['원형', '하양'],
 ['원형', '하양'],
 ['장방형', '하양'],
 ['원형', '하양'],
 ['장방형', '연두'],
 ['사각형', '노랑'],
 ['타원형', '노랑'],
 ['장방형', '하양'],
 ['원형', '하양'],
 ['원형', '하양'],
 ['장방형', '갈색'],
 ['원형', '하양'],
 ['타원형', '노랑'],
 ['원형', '노랑'],
 ['타원형', '파랑'],
 ['장방형', '하양'],
 ['장방형', '파랑'],
 ['원형', '하양'],
 ['장방형', '연두'],
 ['원형', '하양'],
 ['장방형', '파랑'],
 ['원형', '주황'],
 ['장방형', '초록'],
 ['원형', '주황'],
 ['타원형', '노랑'],
 ['오각형', '분홍'],
 ['타원형', '하양'],
 ['장방형', '갈색, 투명'],
 ['타원형', '하양'],
 ['원형', '하양'],
 ['장방형', '빨강'],
 ['타원형', '노랑'],
 ['장방형', '하양'],
 ['장방형', '분홍'],
 ['타원형', '하양'],
 ['원형', '하양'],
 ['원형', '노랑'],
 ['원형', '하양'],
 ['마름모형', '하양'],
 ['원형', '하양'],
 ['원형', '하양'],
 ['기타', '초록'],
 ['원형', '초록'],
 ['장방형', '갈색, 투명'],
 ['타원형', '빨강'],
 ['원형', '하양'],
 ['원형', '노랑'],
 ['원형', '하양'],
 ['타원형', '주황'],
 ['원형', '하양'],
 ['장방형', '주황'],
 ['기타', '하양'],
 ['원형', '하양'],
 ['원형', '파랑'],
 ['원형', '하양'],
 ['원형', '하양'],
 ['장방형', '회색'],

In [ ]:
#png 파일 있으면 jpg 로 변환해주기

for sampleimg in listdir :
  if os.path.splitext(sampleimg)[-1] == '.png':
    im = Image.open(data_dir + '/'+sampleimg)
    im = im.convert("RGB")
    im.save(data_dir + '/'+'%s.jpg'%os.path.splitext(sampleimg)[0])
    print(sampleimg)
    os.remove(data_dir + '/'+sampleimg)



닥터베아제정.png
포리부틴정.png
프론크논캡슐.png
더블자임정.png
다우제큐정.png
유한세파클러캡슐.png
굿스펜연질캡슐.png
맥시쿨펜연질캡슐.png
타이레놀정500밀리그람.png
오프노즈연질캡슐.png
화이투벤씨플러스캡슐.png
알마겔정.png
락토바이장용캡슐.png
엔시트라세미정.png
헤토마신캡슐.png
브레핀에스연질캡슐.png
펜스톱연질캡슐.png
푸푸에프연질캡슐.png
콜드펜노즈연질캡슐.png
인브로정.png
둘코락스에스장용정.png
가스모틴정5밀리그램.png
